In [0]:
-- 🔹 lake Customers Table
CREATE OR REFRESH LIVE TABLE lake_schema.customers
COMMENT "lake customer data"
AS SELECT * FROM VALUES
  (1, "Alice", "US"),
  (2, "Bob", "DE"),
  (3, "Charlie", "FR")
AS data(customer_id, name, country);

-- 🔹 lake Products Table
CREATE OR REFRESH LIVE TABLE lake_schema.products
COMMENT "lake product data"
AS SELECT * FROM VALUES
  (101, "Laptop", "Electronics"),
  (102, "Chair", "Furniture"),
  (103, "Pen", "Stationery")
AS data(product_id, product_name, category);

-- 🔹 lake Sales Table
CREATE OR REFRESH LIVE TABLE lake_schema.sales
COMMENT "lake sales data"
AS SELECT * FROM VALUES
  (1, 101, "2024-01-01", 1200),
  (2, 103, "2024-01-02", 5),
  (3, 102, "2024-01-03", 100)
AS data(customer_id, product_id, date, amount);

In [0]:
-- 🌟 Dimension Table: Customers
CREATE OR REFRESH LIVE TABLE hub_schema.dim_customers(
  customer_id INTEGER PRIMARY KEY,
  customer_name STRING,
  country STRING
)
COMMENT "Customer dimension with PK"
TBLPROPERTIES (
  'quality' = 'gold'
)
AS SELECT 
  customer_id,
  name AS customer_name,
  country
FROM lake_schema.customers;

-- 🌟 Dimension Table: Products
CREATE OR REFRESH LIVE TABLE hub_schema.dim_products(
  product_id INTEGER PRIMARY KEY,
  name STRING,
  category STRING
)
COMMENT "Product dimension with PK"
TBLPROPERTIES (
  'quality' = 'gold'
)
AS SELECT 
  product_id,
  product_name AS name,
  category
FROM lake_schema.products;

-- 📦 Fact Table with joins + FK to dimensions
CREATE OR REFRESH LIVE TABLE hub_schema.fact_sales_star(
  row_key STRING PRIMARY KEY,
  customer_id INTEGER,
  product_id INTEGER,
  --customer_id INTEGER FOREIGN KEY REFERENCES hub_schema.dim_customers(customer_id),
  --product_id INTEGER FOREIGN KEY REFERENCES hub_schema.dim_products(product_id),
  customer_name STRING,
  product_name STRING,
  category STRING,
  amount DOUBLE,
  date DATE
)
COMMENT "Sales fact table joined with customer and product dimensions"
TBLPROPERTIES (
  'quality' = 'gold'
)
AS SELECT 
  MD5(CONCAT_ws("|",sales.customer_id, sales.product_id)) as row_key,
  sales.customer_id,
  sales.product_id,
  cust.customer_name,
  prod.name as product_name,
  prod.category,
  CAST(sales.amount AS DOUBLE) AS amount,
  TO_DATE(sales.date) AS date
FROM lake_schema.sales sales
JOIN hub_schema.dim_customers cust ON sales.customer_id = cust.customer_id
JOIN hub_schema.dim_products prod ON sales.product_id = prod.product_id;
